# Stable diffusion experiments

## Env. setup

In [ ]:
! nvidia-smi

### Installations

In [ ]:
! pip install -q diffusers
! pip install -q transformers accelerate
! pip install -q torch torchvision -U
! pip install -q xformers git+https://github.com/huggingface/accelerate.git
! pip install -q opencv-contrib-python
! pip install -q controlnet_aux

### Imports

In [ ]:
import os
import cv2
import torch
import tempfile

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
from diffusers.utils import load_image

In [ ]:
torch.__version__

### Global Env. variables

In [ ]:
load_loc = 'templates'
save_loc = 'outputs'

clear_workspace = False

In [ ]:
if not os.path.exists(load_loc):
    os.makedirs(load_loc)

if not os.path.exists(save_loc):
  os.makedirs(save_loc)

### Stable diffusion setup

In [ ]:
model_id_or_path = 'Ojimi/anime-kawai-diffusion'
# torch_dtype = torch.float32

# model_id_or_path = "runwayml/stable-diffusion-v1-5"
torch_dtype=torch.float16

In [ ]:
# Stable diffusion pipe
sd_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype = torch_dtype)
sd_pipe = sd_pipe.to('cuda')

### ControlNet setup

In [ ]:
controlnet = ControlNetModel.from_pretrained('lllyasviel/sd-controlnet-canny', torch_dtype = torch_dtype)
cn_pipe = StableDiffusionControlNetPipeline.from_pretrained(model_id_or_path, controlnet = controlnet, torch_dtype = torch_dtype)

In [ ]:
cn_pipe.scheduler = UniPCMultistepScheduler.from_config(cn_pipe.scheduler.config)

In [ ]:
cn_pipe.enable_model_cpu_offload()

In [ ]:
try:
  cn_pipe.enable_xformers_memory_efficient_attention()
except:
  pass

### Methods

In [ ]:
def generate_temp_name():
  temp = tempfile.NamedTemporaryFile()
  temp_name = temp.name.split('/')[-1]

  return temp_name

In [ ]:
def process(input_image, pipe, prompt, **kwargs):
  output_images = pipe(image = input_image, prompt = prompt, **kwargs).images

  return output_images

In [ ]:
def load(image_name, image_path = load_loc, shape = (400, 500), format = 'RGB'):
  image_path = os.path.join(image_path, image_name)
  image = Image.open(image_path).convert(format).resize(shape)

  return image

In [ ]:
def save(images, save_loc = save_loc):
  temp_name = generate_temp_name()
  image_loc = os.path.join(save_loc, f'{temp_name}.png')

  images[0].save(image_loc)

  return image_loc

In [ ]:
def canny_image(image, threshold):
  image = np.array(image)
  low_threshold, high_threshold = threshold

  image = cv2.Canny(image, low_threshold, high_threshold)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis = 2)

  canny = Image.fromarray(image)

  return canny

In [ ]:
def plot(images, title):
  plt.title(title)
  plt.imshow(images[0])
  plt.axis('off')
  plt.show()

## Experiments

### Stable Diffusion

In [ ]:
image_name = 'img_6.jpg'

input_image = load(image_name, shape = (400, 600))
plot([input_image], title = 'Original')

In [ ]:
prompt = "Upper body, official art, slim, beautiful, aesthetic, looking at viewer, dynamic pose, perfect angle, long hair, petite, beautiful skin, background, beautiful detailed eyes, muted color, depth of field, volumetric lighting, reflection, finest detail, ultra detailed, perfect face, epic proportion, epic composition, professional work"

In [ ]:
negative_prompt = 'lowres, bad anatomy'

In [ ]:
images = process(input_image, sd_pipe, prompt, negative_prompt = negative_prompt, strength = 0.75, guidance_scale = 3.5)
output_image = save(images)

print(output_image)
plot(images, title = output_image.split('/')[-1])

### ControlNet

In [ ]:
# input_image = load_image(
#     "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
# )
# input_image

In [ ]:
threshold = (100, 200)

image = canny_image(input_image, threshold)
image

In [ ]:
images = process(image, cn_pipe, prompt, negative_prompt = negative_prompt, num_inference_steps = 20)
output_image = save(images)

print(output_image)
plot(images, title = output_image.split('/')[-1])

In [ ]:
if clear_workspace:
  ! rm outputs/*

## Resources